<a href="https://colab.research.google.com/github/Computer-CGuy/MusgraveTextureNeuralRenderer/blob/models/SKIN_CANCER_%7C_NeuralRenderer_0.13_L1Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch  
import matplotlib.pyplot as plt  
import numpy as np  
import torch.nn.functional as func
import torch.nn.functional as F
import PIL.ImageOps  
from torch import nn  
from torchvision import datasets,transforms   
import requests  
from PIL import Image  
import torchvision.models as models
import os
import json
from torch.utils.data import Dataset, DataLoader, Subset
import cv2
from google.colab.patches import cv2_imshow
import torchvision
import pickle

In [2]:
net = models.resnext50_32x4d(pretrained=True).cuda()

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


In [3]:
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [4]:
net.fc = nn.Sequential(
          nn.Linear(2048,1024),
          nn.LeakyReLU(),
          nn.Linear(1024,512),
          nn.LeakyReLU(),
          nn.Linear(512,4),
          nn.LeakyReLU()
        ).cuda()

In [5]:
from PIL import Image

In [7]:
!ls "/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries" | wc -l

10251


In [7]:
# with open("/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries/out.pkl","rb") as f:
#     l = (pickle.load(f))

In [11]:
# !ls "/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries/" | grep ".*pkl"

out.pkl


In [10]:
# !ls "/content/drive/My Drive/*/" | wc -l

ls: cannot access '/content/drive/My Drive/*/': No such file or directory
0


In [8]:
class MusgraveDataset(Dataset):
    def __init__(self,transforms):
        with open("/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries/out.pkl","rb") as f:
            self.l = (pickle.load(f))
        self.transforms = transforms
    def __getitem__(self, index):
        # print(index)
        l = self.l
        file_name = l[index]["name"]
        file_name = "/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries/"+file_name
        img = Image.open(file_name)
        # img = cv2.imread(file_name)
        # img = np.transpose(img,(2,0,1))
        # transforms.Compose([
            
        # ])
        return ((self.transforms(img),torch.Tensor(l[index]["values"])))
    
    def __len__(self):
        return len(self.l)

In [ ]:
# !ls "./drive/My Drive/MusgraveTextureNeuralRenderer/Generator/*.pkl"

In [ ]:
# import pickle
# with open("./drive/My Drive/MusgraveTextureNeuralRenderer/out.pkl","rb") as f:
#     l = (pickle.load(f))

In [9]:
transform = transforms.Compose([torchvision.transforms.Resize((224,224)),transforms.ToTensor()])
train = MusgraveDataset(transforms = transform)
train_loader = DataLoader(dataset=train, batch_size=10, shuffle=True, num_workers=2)

In [28]:
# train[0][0].shape

torch.Size([3, 224, 224])

In [10]:
import torch.optim as optim

# criterion = nn.MSELoss(reduction='sum')
criterion = nn.L1Loss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

# optimizer = optim.SGD(render.parameters(), lr=0.001, momentum=0.9)

In [17]:
def show(img):
    img = img.detach().cpu().numpy()[0]
    img  = np.transpose(img,(1,2,0))
    print(img.shape)
    cv2_imshow(img)

In [ ]:
# train.l

In [ ]:
# inputs.shape

In [11]:
loss_n = None
for epoch in range(3):
    for i,data in enumerate(train_loader):
        inputs, labels = data[0].cuda(),data[1].cuda()
        labels[:,0]/=12
        labels[:,1]/=16
        labels[:,2]/=4
        labels[:,3]/=3
        # labels/=255
        optimizer.zero_grad()
        
        output = net(inputs)
        
        loss = criterion(output, labels)
        try:
            loss_n+=loss
        except:
            loss_n = loss
        
        if(i%10==9):
            loss_n.backward()
            optimizer.step()
            # print(loss.item())
            print("Epoch:- %d Loss:- %0.5f i: %d"%(epoch,loss_n.item()/10,i))
            # print(output,inputs)
            loss_n = None
        
        

Epoch:- 0 Loss:- 0.49121 i: 9
Epoch:- 0 Loss:- 0.42338 i: 19
Epoch:- 0 Loss:- 0.38766 i: 29
Epoch:- 0 Loss:- 0.33608 i: 39
Epoch:- 0 Loss:- 0.31329 i: 49
Epoch:- 0 Loss:- 0.28361 i: 59
Epoch:- 0 Loss:- 0.25450 i: 69
Epoch:- 0 Loss:- 0.24168 i: 79
Epoch:- 0 Loss:- 0.24739 i: 89
Epoch:- 0 Loss:- 0.22522 i: 99
Epoch:- 0 Loss:- 0.21061 i: 109
Epoch:- 0 Loss:- 0.23356 i: 119
Epoch:- 0 Loss:- 0.20752 i: 129
Epoch:- 0 Loss:- 0.22359 i: 139
Epoch:- 0 Loss:- 0.21530 i: 149
Epoch:- 0 Loss:- 0.20745 i: 159
Epoch:- 0 Loss:- 0.20610 i: 169
Epoch:- 0 Loss:- 0.21721 i: 179
Epoch:- 0 Loss:- 0.20480 i: 189
Epoch:- 0 Loss:- 0.20825 i: 199
Epoch:- 0 Loss:- 0.19272 i: 209
Epoch:- 0 Loss:- 0.19495 i: 219
Epoch:- 0 Loss:- 0.20270 i: 229
Epoch:- 0 Loss:- 0.19499 i: 239
Epoch:- 0 Loss:- 0.20895 i: 249
Epoch:- 0 Loss:- 0.17998 i: 259
Epoch:- 0 Loss:- 0.20747 i: 269
Epoch:- 0 Loss:- 0.19681 i: 279
Epoch:- 0 Loss:- 0.19558 i: 289
Epoch:- 0 Loss:- 0.19280 i: 299
Epoch:- 0 Loss:- 0.18277 i: 309
Epoch:- 0 Loss:- 0.

In [17]:
print("Epoch:- %d Loss:- %0.3f i: %d"%(epoch,loss_n.item()/10,i))

Epoch:- 0 Loss:- 0.178 i: 13


In [12]:
torch.save(net, "/content/drive/My Drive/MusgraveTextureNeuralRenderer/model3.pth")

In [13]:
output

tensor([[ 5.0979e-01,  6.6193e-01,  9.2161e-01,  5.8289e-01],
        [ 4.2071e-01,  5.3922e-01,  8.4729e-01,  9.7378e-01],
        [ 4.5115e-01,  1.3823e-01,  6.7447e-01,  9.7169e-01],
        [ 6.2531e-01,  4.7036e-01, -8.9519e-04,  5.0912e-01],
        [ 4.1678e-01,  1.3556e-01,  8.3252e-01,  3.0861e-01],
        [ 6.7195e-01,  1.4252e-01,  7.6647e-01, -2.2507e-03],
        [ 4.9573e-01,  5.5175e-01,  4.8187e-01,  2.8703e-01],
        [ 6.2531e-01,  4.7036e-01, -8.9519e-04,  5.0912e-01],
        [ 6.2531e-01,  4.7036e-01, -8.9519e-04,  5.0912e-01],
        [ 4.7158e-01,  4.2530e-01,  2.5933e-01,  4.2073e-01]], device='cuda:0',
       grad_fn=<LeakyReluBackward0>)

In [14]:
labels

tensor([[0.3333, 0.7500, 1.0000, 0.6667],
        [1.0000, 0.5625, 1.0000, 1.0000],
        [0.7500, 0.1875, 0.5000, 1.0000],
        [0.9167, 0.0625, 0.0000, 1.0000],
        [0.9167, 0.1875, 0.7500, 0.3333],
        [0.5833, 0.1875, 0.7500, 0.0000],
        [0.2500, 0.6250, 0.5000, 0.3333],
        [0.1667, 0.8750, 0.0000, 1.0000],
        [0.7500, 0.8750, 0.0000, 1.0000],
        [0.2500, 0.4375, 0.2500, 0.0000]], device='cuda:0')

In [ ]:
torch.save(render.state_dict(), "./drive/My Drive/Skin Cancer/RenderResNet.pth")

In [ ]:
torch.save(render.state_dict(), "./drive/My Drive/Skin Cancer/AlexNet.pth")

In [ ]:
def norm(out):
    out-=out.mean()
    out/=out.max()
    return out

In [ ]:
inputs.shape,output.shape

(torch.Size([4, 4]), torch.Size([4, 1000]))

In [ ]:
NetEpochs = 10000
start, stop, step = 7,10,1
Dataset = [x for x in range(1000)]
# Dataset = [0,1]
# Dataset = [0]
done = False
for data in Dataset:
  
  tdata = torch.Tensor([p]).cuda()
  X = model(tdata)
  # print(X)t
  X_CNN = cnn(X)
  # X = X*(255/X.mean())

  X_CNN2 = cnn2(X)
  
  img = process_image(X)
  # print(data,end=" ")
  # print("[ AlexNet "+str(dat[int(torch.argmax(X_CNN).detach().cpu().numpy())]),end="] ")
  # print("[ ResNet "+str(dat[int(torch.argmax(X_CNN2).detach().cpu().numpy())]),end="]")
  
  # cv2_imshow(img)
  cv2.imwrite("dog.jpg",img)
  os.system("python gradcam.py --image-path=dog.jpg --use-cuda")
  cp(data)
  updateJson(data,X_CNN,X_CNN2)
  if((data%10)==0):
    print(data)
  # break
  # imgR = cv2.imread("cam.jpg")
  # cv2_imshow(imgR)
